# Part 1: Data Extraction and Database Design

In [2]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import re
import time

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"

# fetching the content of the page
response = requests.get(url)
if response.status_code == 200:
    page_content = response.text
    print("Successful success (code 200)! ! !")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
    page_content = ""


Successful success (code 200)! ! !


In [11]:
soup = BeautifulSoup(page_content, 'html.parser')
# for the first table in the page
table = soup.find('table', {'class': 'wikitable'})
if table:
    rows = table.find_all('tr')
else:
    print("No table found on the page.")
    rows = []

films_data = []

for row in rows[1:]:
    cols = row.find_all(['td', 'th'])
    if len(cols) < 5:
        continue

    title_col = cols[2].find('a')
    title = title_col.get_text(strip=True)
    release_year = cols[4].get_text(strip=True)
    box_office = cols[3].get_text(strip=True)
    film_link = title_col['href'] if title_col else None

    release_year = int(release_year)
    box_office = int(box_office.split("$")[1].replace(",", ""))  # from $123,456 to 123456

    # # remove any references in brackets
    # title = re.sub(r'\[.*?\]', '', title)
    # box_office = re.sub(r'\[.*?\]', '', box_office)

    films_data.append({
        'title': title,
        'release_year': release_year,
        'box_office': box_office,
        'film_link': film_link
    })


In [12]:
films_data

[{'title': 'Avatar',
  'release_year': 2009,
  'box_office': 2923706026,
  'film_link': '/wiki/Avatar_(2009_film)'},
 {'title': 'Avengers: Endgame',
  'release_year': 2019,
  'box_office': 2797501328,
  'film_link': '/wiki/Avengers:_Endgame'},
 {'title': 'Avatar: The Way of Water',
  'release_year': 2022,
  'box_office': 2320250281,
  'film_link': '/wiki/Avatar:_The_Way_of_Water'},
 {'title': 'Titanic',
  'release_year': 1997,
  'box_office': 2257844554,
  'film_link': '/wiki/Titanic_(1997_film)'},
 {'title': 'Star Wars: The Force Awakens',
  'release_year': 2015,
  'box_office': 2068223624,
  'film_link': '/wiki/Star_Wars:_The_Force_Awakens'},
 {'title': 'Avengers: Infinity War',
  'release_year': 2018,
  'box_office': 2048359754,
  'film_link': '/wiki/Avengers:_Infinity_War'},
 {'title': 'Ne Zha 2',
  'release_year': 2025,
  'box_office': 1940794560,
  'film_link': '/wiki/Ne_Zha_2'},
 {'title': 'Spider-Man: No Way Home',
  'release_year': 2021,
  'box_office': 1922598800,
  'film_lin

In [37]:
base_url = "https://en.wikipedia.org"

# extract director(s) and country from individual film page
def get_additional_details(film_link):
    # access by url
    film_url = base_url + film_link
    response = requests.get(film_url)
    if response.status_code != 200:
        print(f"Failed to retrieve the page for {film_url}. Status code: {response.status_code}")
        return None, None

    film_soup = BeautifulSoup(response.text, 'html.parser')

    director = None
    country = None
    # retrieving infobox on the side
    infobox = film_soup.find('table', {'class': 'infobox'})
    if infobox:
        director_row = infobox.find('th', string='Directed by')
        if director_row:
            director = director_row.find_next_sibling('td').get_text(separator=', ', strip=True)
            # because citations look awful in parsed data
            director = ', '.join([d for d in director.split(", ") if d.replace(" ", "").replace(".", "").isalpha()])

        country_row = infobox.find('th', string='Country')
        if country_row:
            country = country_row.find_next_sibling('td').get_text(separator=', ', strip=True)
        if not country:
            country_row = infobox.find('th', string='Countries')
            if country_row:    # taking only the first country (was told to do so in the chat)
                country = country_row.find_next_sibling('td').get_text(separator=', ', strip=True).split(",")[0]

    return director, country

for film in films_data:
    director, country = get_additional_details(film['film_link'])
    film['director'] = director
    film['country'] = country

In [38]:
films_data

[{'title': 'Avatar',
  'release_year': 2009,
  'box_office': 2923706026,
  'film_link': '/wiki/Avatar_(2009_film)',
  'director': 'James Cameron',
  'country': 'United Kingdom'},
 {'title': 'Avengers: Endgame',
  'release_year': 2019,
  'box_office': 2797501328,
  'film_link': '/wiki/Avengers:_Endgame',
  'director': 'Anthony Russo, Joe Russo',
  'country': 'United States'},
 {'title': 'Avatar: The Way of Water',
  'release_year': 2022,
  'box_office': 2320250281,
  'film_link': '/wiki/Avatar:_The_Way_of_Water',
  'director': 'James Cameron',
  'country': 'United States'},
 {'title': 'Titanic',
  'release_year': 1997,
  'box_office': 2257844554,
  'film_link': '/wiki/Titanic_(1997_film)',
  'director': 'James Cameron',
  'country': 'United States'},
 {'title': 'Star Wars: The Force Awakens',
  'release_year': 2015,
  'box_office': 2068223624,
  'film_link': '/wiki/Star_Wars:_The_Force_Awakens',
  'director': 'J. J. Abrams',
  'country': 'United States'},
 {'title': 'Avengers: Infinity 

In [43]:
# either connect or create db
conn = sqlite3.connect('films.db')
cursor = conn.cursor()

# Create the films table
cursor.execute('''
CREATE TABLE IF NOT EXISTS films (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    release_year INTEGER,
    director TEXT,
    box_office TEXT,
    country TEXT
)
''')

films_to_insert = [
    (film['title'], film['release_year'], film['director'], film['box_office'], film['country'])
    for film in films_data
]

# insert data into the table
cursor.executemany('''
INSERT INTO films (title, release_year, director, box_office, country)
VALUES (?, ?, ?, ?, ?)
''', films_to_insert)

conn.commit()

# view the table we created
cursor.execute("SELECT * FROM films")
rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()

(1, 'Avatar', 2009, 'James Cameron', '2923706026', 'United Kingdom')
(2, 'Avengers: Endgame', 2019, 'Anthony Russo, Joe Russo', '2797501328', 'United States')
(3, 'Avatar: The Way of Water', 2022, 'James Cameron', '2320250281', 'United States')
(4, 'Titanic', 1997, 'James Cameron', '2257844554', 'United States')
(5, 'Star Wars: The Force Awakens', 2015, 'J. J. Abrams', '2068223624', 'United States')
(6, 'Avengers: Infinity War', 2018, 'Anthony Russo, Joe Russo', '2048359754', 'United States')
(7, 'Ne Zha 2', 2025, 'Jiaozi', '1940794560', 'China')
(8, 'Spider-Man: No Way Home', 2021, 'Jon Watts', '1922598800', 'United States')
(9, 'Inside Out 2', 2024, 'Kelsey Mann', '1698863816', 'United States')
(10, 'Jurassic World', 2015, 'Colin Trevorrow', '1671537444', 'United States')
(11, 'The Lion King', 2019, 'Jon Favreau', '1656943394', 'United States')
(12, 'The Avengers', 2012, 'Joss Whedon', '1518815515', 'United States')
(13, 'Furious 7', 2015, 'James Wan', '1515341399', 'United States')


# Part 2: Web Page Development and GitHub Pages Hosting

In [46]:
import sqlite3
import json

conn = sqlite3.connect("films.db")
cursor = conn.cursor()

cursor.execute("SELECT title, release_year, director, box_office, country FROM films;")
films = cursor.fetchall()

# tuples to a list of dictionaries
films_list = [{"title": row[0], "release_year": row[1], "director": row[2], "box_office": row[3], "country": row[4]} for row in films]

with open("films.json", "w", encoding="utf-8") as f:
    json.dump(films_list, f, indent=4)

conn.close()